In [16]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np

##### Read image data using generators

In [10]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,
    shear_range=0.1,zoom_range=0.10,horizontal_flip=True
)
train_generator = datagen.flow_from_directory('dogs_vs_cats/train', target_size=(150,150), 
                                              batch_size=128, class_mode='binary',
                                              )
test_generator = datagen.flow_from_directory('dogs_vs_cats/test', target_size=(150,150), 
                                              batch_size=128, class_mode='binary',
                                              )

Found 19958 images belonging to 2 classes.
Found 4992 images belonging to 2 classes.


In [2]:
train_generator.class_indices

NameError: name 'train_generator' is not defined

##### Train model


In [11]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='valid', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 128, 128, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 64)        0

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=20,
          validation_data=test_generator)

In [ ]:
model.save('cat_vs_dog.h5')

In [2]:
load_model = tf.keras.models.load_model('cat_vs_dog.h5')

##### Testing model

In [3]:
load_model.evaluate(test_generator)

NameError: name 'test_generator' is not defined

In [ ]:
plt.plot(history.history['loss'], c='b', label='train')
plt.plot(history.history['val_loss'], c='r', label='test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], c='b', label='train')
plt.plot(history.history['val_accuracy'], c='r', label='test')
plt.legend()
plt.show()

In [30]:
test_cat = cv2.imread('test\cat.jpg')
test_dog = cv2.imread('test\dog.jpg')

test_cat.shape, test_dog.shape

((194, 259, 3), (275, 183, 3))

In [31]:
test_cat = cv2.resize(test_cat, (150, 150))
test_dog = cv2.resize(test_dog, (150, 150))
test_cat.shape, test_dog.shape

((150, 150, 3), (150, 150, 3))

In [32]:
test_cat = np.expand_dims(test_cat, axis=0)
test_dog = np.expand_dims(test_dog, axis=0)
test_cat.shape, test_dog.shape

((1, 150, 150, 3), (1, 150, 150, 3))

In [33]:
test_cat = test_cat / 255.
test_dog = test_dog / 255.

In [39]:
pred = load_model.predict(test_cat)

1/1 [==============================] - 0s 82ms/step


In [40]:
TH = 0.5
1 if pred > TH else 0

0

In [36]:
pred = load_model.predict(test_dog)

1/1 [==============================] - 0s 199ms/step


In [37]:
TH = 0.5
1 if pred > TH else 0

1